In [1]:
import numpy as np
import pints

import chi
from chi.library import ModelLibrary

In [2]:
mechanistic_model = ModelLibrary().one_compartment_pk_model()
mechanistic_model.set_administration(
    compartment='central', amount_var='drug_amount')
error_model = chi.LogNormalErrorModel()
predictive_model = chi.PredictiveModel(mechanistic_model, error_model)


INFO:root:running build_ext
INFO:root:building 'myokit_sim_1_5144858832636078169' extension
INFO:root:creating build
INFO:root:creating build/temp.linux-x86_64-cpython-310
INFO:root:creating build/temp.linux-x86_64-cpython-310/tmp
INFO:root:creating build/temp.linux-x86_64-cpython-310/tmp/tmpyj_tatuqmyokit
INFO:root:x86_64-linux-gnu-gcc -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -fPIC -I/usr/local/include -I/opt/local/include -I/home/rumney/Documents/Myleotoxicity/Myleotoxicity-PKPD/venv/lib/python3.10/site-packages/myokit/_sim -I/home/rumney/Documents/Myleotoxicity/Myleotoxicity-PKPD/venv/lib/python3.10/site-packages/myokit/_sim -I/home/rumney/Documents/Myleotoxicity/Myleotoxicity-PKPD/venv/include -I/usr/include/python3.10 -c /tmp/tmpyj_tatuqmyokit/source.c -o build/temp.linux-x86_64-cpython-310/tmp/tmpyj_tatuqmyokit/source.o
INFO:root:creating build/lib.linux-x86_64-cpython-310
INFO:root

In [3]:
times = np.arange(10)
parameters = [1, 1, 1, 0.1]
predictive_model.set_dosing_regimen(2, period=1)
data = predictive_model.sample(
    parameters=parameters, times=times, include_regimen=True)

In [4]:
data

,ID,Time,Observable,Value,Duration,Dose
0,1,0,central.drug_concentration,0.900200,NaN,NaN
0,1,1,central.drug_concentration,0.947165,NaN,NaN
0,1,2,central.drug_concentration,1.124374,NaN,NaN
0,1,3,central.drug_concentration,1.011353,NaN,NaN
0,1,4,central.drug_concentration,1.227973,NaN,NaN
0,1,5,central.drug_concentration,1.239882,NaN,NaN
0,1,6,central.drug_concentration,1.165645,NaN,NaN
0,1,7,central.drug_concentration,1.102739,NaN,NaN
0,1,8,central.drug_concentration,1.279268,NaN,NaN
0,1,9,central.drug_concentration,1.390376,NaN,NaN


In [5]:
mechanistic_model.outputs()

['central.drug_concentration']

In [6]:
problem = chi.ProblemModellingController(mechanistic_model, error_model)
problem.set_data(data)
problem.set_log_prior(pints.UniformLogPrior([0.1]*4, [4]*4))
log_posterior = problem.get_log_posterior()

INFO:root:running build_ext
INFO:root:building 'myokit_sim_4_5144858832636078169' extension
INFO:root:creating build
INFO:root:creating build/temp.linux-x86_64-cpython-310
INFO:root:creating build/temp.linux-x86_64-cpython-310/tmp
INFO:root:creating build/temp.linux-x86_64-cpython-310/tmp/tmpucxc5tonmyokit
INFO:root:x86_64-linux-gnu-gcc -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -fPIC -I/usr/local/include -I/opt/local/include -I/home/rumney/Documents/Myleotoxicity/Myleotoxicity-PKPD/venv/lib/python3.10/site-packages/myokit/_sim -I/home/rumney/Documents/Myleotoxicity/Myleotoxicity-PKPD/venv/lib/python3.10/site-packages/myokit/_sim -I/home/rumney/Documents/Myleotoxicity/Myleotoxicity-PKPD/venv/include -I/usr/include/python3.10 -c /tmp/tmpucxc5tonmyokit/source.c -o build/temp.linux-x86_64-cpython-310/tmp/tmpucxc5tonmyokit/source.o
INFO:root:creating build/lib.linux-x86_64-cpython-310
INFO:root

In [7]:
log_posterior.get_parameter_names()

['central.drug_amount', 'central.size', 'myokit.elimination_rate', 'Sigma log']

In [8]:
population_model = chi.ComposedPopulationModel([
    chi.LogNormalModel(n_dim=3),
    chi.PooledModel()])
pop_predictive_model = chi.PopulationPredictiveModel(
    predictive_model, population_model)

In [9]:
parameters = [1] * 6 + [0.1]
data = pop_predictive_model.sample(
    parameters, times, include_regimen=True, n_samples=3)

In [10]:
problem = chi.ProblemModellingController(mechanistic_model, error_model)
problem.set_population_model(population_model)
problem.set_data(data)
problem.set_log_prior(pints.UniformLogPrior([0.1]*7, [4]*7))
log_posterior = problem.get_log_posterior()
log_posterior.get_parameter_names()

INFO:root:running build_ext
INFO:root:building 'myokit_sim_6_5144858832636078169' extension
INFO:root:creating build
INFO:root:creating build/temp.linux-x86_64-cpython-310
INFO:root:creating build/temp.linux-x86_64-cpython-310/tmp
INFO:root:creating build/temp.linux-x86_64-cpython-310/tmp/tmpe4ibdsi9myokit
INFO:root:x86_64-linux-gnu-gcc -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -fPIC -I/usr/local/include -I/opt/local/include -I/home/rumney/Documents/Myleotoxicity/Myleotoxicity-PKPD/venv/lib/python3.10/site-packages/myokit/_sim -I/home/rumney/Documents/Myleotoxicity/Myleotoxicity-PKPD/venv/lib/python3.10/site-packages/myokit/_sim -I/home/rumney/Documents/Myleotoxicity/Myleotoxicity-PKPD/venv/include -I/usr/include/python3.10 -c /tmp/tmpe4ibdsi9myokit/source.c -o build/temp.linux-x86_64-cpython-310/tmp/tmpe4ibdsi9myokit/source.o
INFO:root:creating build/lib.linux-x86_64-cpython-310
INFO:root

['central.drug_amount',
 'central.size',
 'myokit.elimination_rate',
 'central.drug_amount',
 'central.size',
 'myokit.elimination_rate',
 'central.drug_amount',
 'central.size',
 'myokit.elimination_rate',
 'Log mean central.drug_amount',
 'Log mean central.size',
 'Log mean myokit.elimination_rate',
 'Log std. central.drug_amount',
 'Log std. central.size',
 'Log std. myokit.elimination_rate',
 'Pooled Sigma log']

In [34]:
data

,ID,Time,Observable,Value,Duration,Dose
0,1,0.0,central.drug_concentration,0.610561,NaN,NaN
1,2,0.0,central.drug_concentration,0.298841,NaN,NaN
2,3,0.0,central.drug_concentration,11.664585,NaN,NaN
3,1,1.0,central.drug_concentration,0.685605,NaN,NaN
4,2,1.0,central.drug_concentration,0.191078,NaN,NaN
5,3,1.0,central.drug_concentration,1.849333,NaN,NaN
6,1,2.0,central.drug_concentration,0.767075,NaN,NaN
7,2,2.0,central.drug_concentration,0.168279,NaN,NaN
8,3,2.0,central.drug_concentration,0.549639,NaN,NaN
9,1,3.0,central.drug_concentration,0.768302,NaN,NaN
